## Домашняя работа 12 - Продвинутый pandas
### Задание 1
Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:

- если источник traffic_source равен yandex или google, то в source_type ставится organic
- для источников paid и email из России - ставим ad
- для источников paid и email не из России - ставим other
- все остальные варианты берем из traffic_source без изменений

In [1]:
import pandas as pd

In [3]:
log = pd.read_csv('.\\HW-12 data\\visit_log.csv', sep=';')


In [4]:
def get_source_type(row):
    if row['traffic_source'] in ['yandex','google']: 
        return 'organic'
    elif row['traffic_source'] in ['paid','email']: 
        if row['region'] == 'Russia': return 'ad'
        else: return 'other'
    else: return row['traffic_source']

In [5]:
%%time

log['source_type'] = log.apply(get_source_type, axis=1) 
log.head()

Wall time: 369 ms


,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic


In [6]:
%%time

log.loc[log.traffic_source.isin(['yandex','google']),'source_type2'] = 'organic'
log.loc[log.traffic_source.isin(['paid','email'])&log.region.str.contains('Russia'),'source_type2'] = 'ad'
log.loc[log.traffic_source.isin(['paid','email'])&(~log.region.isin(['Russia'])),'source_type2'] = 'other'
log.loc[~log.traffic_source.isin(['yandex','google','paid','email']),'source_type2'] = log.traffic_source
log.head(5)


Wall time: 38.8 ms


,timestamp,visit_id,url,region,user_id,traffic_source,source_type,source_type2
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic,organic


In [7]:
log.loc[log.source_type!=log.source_type2] 

,timestamp,visit_id,url,region,user_id,traffic_source,source_type,source_type2


### Задание 2
В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:

- Прочитайте содержимое файла с датафрейм
- Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном


In [8]:
URLs = pd.read_csv('.\\HW-12 data\\URLs.txt')

In [13]:
URLs[URLs.url.str.contains(r'(?=\d{4,9}\-)[\w-]+')].head()

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...


### Задание 3
Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [14]:
ratings = pd.read_csv('.\\HW-12 data\\ml-latest-small\\ratings.csv.')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [16]:
ratings_agg = (ratings[['userId','rating','timestamp']]
        .groupby(['userId'])
        .agg({'rating': {'count'},'timestamp': {'max','min'}})
        )
ratings_agg.head()




rating   timestamp            
        count         min         max
userId                               
1          20  1260759108  1260759205
2          76   835355395   835356246
3          51  1298861589  1298932787
4         204   949778714   949982274
5         100  1163373044  1163375145

In [49]:
def LifeTime(row):
    delta = row['timestamp']['max']-row['timestamp']['min']
    result = (delta/(24*60*60))
    return result

In [51]:
ratings_filt = ratings_agg[(ratings_agg['rating']['count']>100)]
ratings_filt['LifeTime'] = ratings_filt.apply(LifeTime,axis=1) 
ratings_filt.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


rating   timestamp                 LifeTime
        count         min         max             
userId                                            
4         204   949778714   949982274     2.356019
8         116  1154389340  1154474527     0.985961
15       1700   997937239  1469330735  5455.943241
17        363  1127468587  1127476640     0.093206
19        423   855190091   855195373     0.061134

In [53]:
ratings_filt['LifeTime'].mean() # среднее значение в днях

463.89476214829165

### Задание 4

Дана статистика услуг перевозок клиентов компании по типам (см. файл “Python_13_join.ipynb” в разделе Материалы для лекции «Продвинутый pandas» ---- Ноутбуки к лекции «Продвинутый pandas»).
Необходимо сформировать две таблицы:

- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [55]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

In [56]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

In [57]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

In [58]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

In [65]:
all_comp = rzd.merge(auto,how='outer').merge(air,how='outer').fillna(0)
all_comp

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [66]:
with_adr= all_comp.merge(client_base,how='outer').fillna(0)
with_adr

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,0.0,0.0,Комсомольская 4
1,112,2810.0,0.0,0.0,Энтузиастов 8а
2,113,10283.0,57483.0,0.0,Левобережная 1а
3,114,5774.0,83.0,0.0,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,0.0,4834.0,4.0,Строителей 18
6,117,0.0,98.0,13.0,Панфиловская 33
7,118,0.0,0.0,173.0,Мастеркова 4
